# Optimising Model Parameters

Now that we have a model and data it's time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters, and optimizes these parameters using gradient descent. For a more detailed walkthrough of this process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

## Prerequisite Code

We load the code from the previous lessons on **Datasets and DataLoaders** and **Build the Neural Network**.

In [3]:
training_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform = Compose([ 
      ToTensor(),
      Lambda(lambda y: y.flatten())
  ])
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=Compose([ 
      ToTensor(),
      Lambda(lambda y: y.flatten())
  ])
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class MyNeuralNetwork(nn.Module):
  def __init__(self):
    super(MyNeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(28*28, 512)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(512, 512)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(512, 10)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.relu2(x)
    x = self.fc3(x)
    return x

model = MyNeuralNetwork()

## Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates (read more about [hyperparameter](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) tuning)

We define the following hyperparameters for training:
* **Number of Epochs** - the number times to iterate over the dataset

* **Batch Size** - the number of data samples propagated through the network before the parameters are updated

* **Learning Rate** - how much to update models parameters at each batch/epoch.Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimisation Loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
* **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.

* **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

Let's briefly familiarize ourselves with some of the concepts used in the training loop.

### Loss Function


When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include ```nn.MSELoss``` (Mean Square Error) for regression tasks, and [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines ```nn.LogSoftmax``` and ```nn.NLLLoss```.

We pass our model's output logits to ```nn.CrossEntropyLoss```, which will normalize the logits and compute the prediction error.

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimiser




Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the ```optimizer``` object. Here, we use the SGD optimizer; additionally, there are many [different optimizers](https://pytorch.org/docs/stable/optim.html) available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
* Call ```optimizer.zero_grad()``` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.



* Backpropagate the prediction loss with a call to ```loss.backward()```. PyTorch deposits the gradients of the loss w.r.t. each parameter.

* Once we have our gradients, we call ```optimizer.step()``` to adjust the parameters by the gradients collected in the backward pass.

## Wrapping it up


We define ```train_loop``` that loops over our optimization code, and ```test_loop``` that evaluates the model's performance against our test data.

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

  # torch save model with torch.save()
  torch.save({'model_weights': model.state_dict()}, 'model.pt')

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to ```train_loop``` and ```test_loop```. Feel free to increase the number of epochs to track the model's improving performance.




In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296042  [   64/60000]
loss: 2.295827  [ 6464/60000]
loss: 2.272335  [12864/60000]
loss: 2.271542  [19264/60000]
loss: 2.259816  [25664/60000]
loss: 2.216730  [32064/60000]
loss: 2.232727  [38464/60000]
loss: 2.195105  [44864/60000]
loss: 2.202378  [51264/60000]
loss: 2.177230  [57664/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.166577 

Epoch 2
-------------------------------
loss: 2.174120  [   64/60000]
loss: 2.172288  [ 6464/60000]
loss: 2.117008  [12864/60000]
loss: 2.137527  [19264/60000]
loss: 2.092700  [25664/60000]
loss: 2.018604  [32064/60000]
loss: 2.061072  [38464/60000]
loss: 1.980174  [44864/60000]
loss: 1.996942  [51264/60000]
loss: 1.937052  [57664/60000]
Test Error: 
 Accuracy: 57.9%, Avg loss: 1.928493 

Epoch 3
-------------------------------
loss: 1.953500  [   64/60000]
loss: 1.934546  [ 6464/60000]
loss: 1.827004  [12864/60000]
loss: 1.872889  [19264/60000]
loss: 1.760499  [25664/60000]
loss: 1.696632  [32064/600

Loading the model is easy!

In [10]:
model = MyNeuralNetwork()

checkpoint = torch.load('model.pt')
model.load_state_dict(checkpoint['model_weights'])

<All keys matched successfully>

# Accessing layer weights and biases

We can also check the weights and biases a specific layer has after it has been trained

In [12]:
model.fc1.weight, model.fc1.bias

(Parameter containing:
 tensor([[ 0.0172, -0.0330, -0.0139,  ..., -0.0307,  0.0187, -0.0117],
         [-0.0344, -0.0109, -0.0038,  ...,  0.0232,  0.0260, -0.0013],
         [ 0.0008,  0.0175, -0.0088,  ..., -0.0121,  0.0317, -0.0088],
         ...,
         [-0.0242,  0.0182, -0.0078,  ...,  0.0314, -0.0098,  0.0131],
         [-0.0107, -0.0216, -0.0345,  ...,  0.0243, -0.0118, -0.0037],
         [-0.0137, -0.0282, -0.0202,  ..., -0.0283, -0.0161,  0.0152]],
        requires_grad=True),
 Parameter containing:
 tensor([ 5.8582e-03,  2.9430e-02, -3.3531e-02,  2.6839e-02, -2.1593e-02,
         -4.1716e-03,  2.4195e-02, -1.6125e-02,  1.6232e-03,  3.6439e-02,
         -2.3043e-02, -1.3096e-02,  2.1101e-02, -1.2119e-02,  2.5329e-02,
         -1.5096e-02, -7.2799e-03,  2.8381e-02, -1.6743e-02, -8.7338e-03,
         -8.8130e-03,  1.0894e-02, -2.4096e-02,  2.7904e-02,  2.3306e-02,
          3.7503e-02,  3.7143e-02, -3.5524e-02,  2.3372e-02,  1.5238e-02,
          3.3955e-02,  2.7311e-02,  3.25

## Freezing a layer prior to training

Now, we can freeze a specific layer, train the model, and verify whether the weights and biases remain unchanged

In [13]:
model = MyNeuralNetwork() # reset the model

In [14]:
model.fc1.requires_grad_(False) # freeze the first linear layer

Linear(in_features=784, out_features=512, bias=True)

In [15]:
for name, param in model.named_parameters():
  print(f'Layer {name} requires grad = {param.requires_grad}')

Layer fc1.weight requires grad = False
Layer fc1.bias requires grad = False
Layer fc2.weight requires grad = True
Layer fc2.bias requires grad = True
Layer fc3.weight requires grad = True
Layer fc3.bias requires grad = True


In [16]:
weights_before_train, bias_before_train = model.fc1.weight, model.fc1.bias

Let's retrain the model

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299551  [   64/60000]
loss: 2.296839  [ 6464/60000]
loss: 2.289687  [12864/60000]
loss: 2.286998  [19264/60000]
loss: 2.279656  [25664/60000]
loss: 2.271916  [32064/60000]
loss: 2.264755  [38464/60000]
loss: 2.253534  [44864/60000]
loss: 2.253704  [51264/60000]
loss: 2.238899  [57664/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.240959 

Epoch 2
-------------------------------
loss: 2.242520  [   64/60000]
loss: 2.241128  [ 6464/60000]
loss: 2.220755  [12864/60000]
loss: 2.229986  [19264/60000]
loss: 2.219749  [25664/60000]
loss: 2.196443  [32064/60000]
loss: 2.208477  [38464/60000]
loss: 2.184458  [44864/60000]
loss: 2.192126  [51264/60000]
loss: 2.171780  [57664/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 2.173741 

Epoch 3
-------------------------------
loss: 2.182100  [   64/60000]
loss: 2.180009  [ 6464/60000]
loss: 2.146249  [12864/60000]
loss: 2.166101  [19264/60000]
loss: 2.150354  [25664/60000]
loss: 2.116339  [32064/600

In [18]:
weights_after_train, bias_after_train = model.fc1.weight, model.fc1.bias

The two weight matrices and bias vectors should be ```True```



In [19]:
torch.equal(weights_after_train, weights_before_train)

True

In [20]:
torch.equal(bias_before_train, bias_after_train)

True

## Initialising layer weights and biases

The default initialisation for each layer follows a Uniform distribution.

We can fine tune the weight initialisation for each layer as we please!

In [23]:
class MyNeuralNetwork(nn.Module):
  def __init__(self):
    super(MyNeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(28*28, 512)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(512, 512)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(512, 10)

    # simple weight initialisation
    # for more complex models
    # please use the function init_weights(m: nn.Module)
    # and, after initialising your model,
    # call model.apply(init_weights)
    # https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.orthogonal_(self.fc2.weight)
    torch.nn.init.constant_(self.fc3.weight, 42)

    torch.nn.init.constant_(self.fc3.bias.data, 15)
    torch.nn.init.normal_(self.fc1.bias.data)
    torch.nn.init.normal_(self.fc2.bias.data)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.relu2(x)
    x = self.fc3(x)
    return x

model = MyNeuralNetwork()

In [24]:
model.fc1.weight, model.fc1.bias

(Parameter containing:
 tensor([[ 5.1275e-02, -1.3068e-03,  1.5896e-02,  ..., -4.9564e-02,
           4.4105e-02, -6.1847e-03],
         [-3.5086e-02,  2.4820e-02,  3.6459e-03,  ..., -3.0692e-02,
           4.9835e-02, -5.1433e-02],
         [ 2.5943e-02, -1.9529e-02,  3.4130e-02,  ..., -5.0271e-02,
           4.1214e-02, -1.1813e-02],
         ...,
         [ 4.3373e-02,  7.4763e-04,  5.6291e-02,  ...,  2.0110e-02,
          -2.9809e-02, -5.5251e-03],
         [-1.8178e-02, -2.9096e-02,  3.7008e-02,  ..., -4.9279e-02,
          -5.8290e-02, -5.4098e-02],
         [ 3.4350e-03, -3.3557e-05,  5.4387e-02,  ...,  3.5376e-02,
           3.5175e-02, -2.4830e-02]], requires_grad=True),
 Parameter containing:
 tensor([-1.6181e-01, -9.2999e-01, -1.2743e+00, -2.8756e-01,  6.1159e-01,
          1.0674e+00, -8.0036e-01, -5.6633e-01,  7.6102e-01,  5.5649e-01,
         -1.5019e-03,  1.8640e+00,  1.0728e-01, -6.4115e-01,  2.2803e-01,
          5.5686e-02, -3.0776e-01, -1.7295e+00, -6.9720e-01,  1.44

In [27]:
model.fc2.weight, model.fc2.bias

(Parameter containing:
 tensor([[ 2.7968e-02,  4.4823e-02,  1.8171e-03,  ..., -3.3453e-02,
          -7.1288e-02, -1.3297e-03],
         [ 5.6843e-02, -5.4169e-04, -3.8008e-02,  ..., -9.9231e-03,
           1.8337e-02,  3.3371e-02],
         [-2.2703e-02,  6.6771e-02,  2.6552e-02,  ...,  2.4233e-02,
          -1.3982e-01, -1.3285e-04],
         ...,
         [-2.9536e-02, -2.0175e-02, -6.3886e-02,  ..., -3.1870e-02,
           1.0092e-01, -1.2971e-04],
         [ 4.4442e-02,  1.2103e-02, -3.8644e-02,  ..., -3.1490e-02,
          -2.4474e-02,  1.5977e-02],
         [ 4.1921e-02, -2.7832e-02, -2.6377e-02,  ..., -1.2528e-02,
          -1.5423e-02, -2.4361e-02]], requires_grad=True),
 Parameter containing:
 tensor([-1.3825e-01, -1.2891e+00,  8.2745e-01,  4.5596e-01,  4.2847e-01,
          2.4569e-01,  1.0405e+00, -5.0403e-01,  2.9479e-01, -1.3895e+00,
         -3.0098e-01,  5.6779e-01,  4.6672e-01,  1.1677e+00,  8.3628e-01,
         -2.0265e+00, -6.0952e-01, -1.4919e+00,  4.2629e-02,  2.69

In [26]:
model.fc3.weight, model.fc3.bias

(Parameter containing:
 tensor([[42., 42., 42.,  ..., 42., 42., 42.],
         [42., 42., 42.,  ..., 42., 42., 42.],
         [42., 42., 42.,  ..., 42., 42., 42.],
         ...,
         [42., 42., 42.,  ..., 42., 42., 42.],
         [42., 42., 42.,  ..., 42., 42., 42.],
         [42., 42., 42.,  ..., 42., 42., 42.]], requires_grad=True),
 Parameter containing:
 tensor([15., 15., 15., 15., 15., 15., 15., 15., 15., 15.], requires_grad=True))

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302585  [   64/60000]
loss: 2.270050  [ 6464/60000]
loss: 2.217766  [12864/60000]
loss: 2.202322  [19264/60000]
loss: 2.116242  [25664/60000]
loss: 2.096732  [32064/60000]
loss: 2.051363  [38464/60000]
loss: 1.985575  [44864/60000]
loss: 1.942873  [51264/60000]
loss: 1.881074  [57664/60000]
Test Error: 
 Accuracy: 62.1%, Avg loss: 1.851601 

Epoch 2
-------------------------------
loss: 1.868556  [   64/60000]
loss: 1.842717  [ 6464/60000]
loss: 1.696359  [12864/60000]
loss: 1.746359  [19264/60000]
loss: 1.543325  [25664/60000]
loss: 1.560077  [32064/60000]
loss: 1.521134  [38464/60000]
loss: 1.448230  [44864/60000]
loss: 1.442772  [51264/60000]
loss: 1.374155  [57664/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 1.339912 

Epoch 3
-------------------------------
loss: 1.380959  [   64/60000]
loss: 1.381377  [ 6464/60000]
loss: 1.193579  [12864/60000]
loss: 1.340475  [19264/60000]
loss: 1.141150  [25664/60000]
loss: 1.195134  [32064/600

In [30]:
model.fc3.weight

Parameter containing:
tensor([[41.9955, 42.0000, 42.0360,  ..., 41.9974, 41.9805, 42.0001],
        [41.9862, 42.0000, 42.0260,  ..., 42.0052, 41.9403, 42.0005],
        [42.0167, 42.0000, 41.9683,  ..., 42.0037, 41.9771, 42.0001],
        ...,
        [41.9908, 42.0000, 42.0164,  ..., 42.0000, 42.0509, 42.0089],
        [41.9858, 42.0000, 42.0124,  ..., 42.0000, 42.0144, 42.0006],
        [41.9936, 42.0000, 41.9179,  ..., 42.0002, 42.1094, 41.9999]],
       requires_grad=True)

# Convolutional Network

In [39]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    # stride = 1, padding = 0, dilation = 1 by default
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()

  def forward(self, x):
    # input size = 1x28x28
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()

## Redefine the transformation of the dataset

In [40]:
training_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform = ToTensor()
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## Train the CNN

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, net, loss_fn, optimizer)
  test_loop(test_dataloader, net, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300425  [   64/60000]
loss: 2.302058  [ 6464/60000]
loss: 2.303045  [12864/60000]
loss: 2.309356  [19264/60000]
loss: 2.283989  [25664/60000]
loss: 2.310388  [32064/60000]
loss: 2.296953  [38464/60000]
loss: 2.303050  [44864/60000]
loss: 2.298905  [51264/60000]
loss: 2.301833  [57664/60000]
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.298695 

Epoch 2
-------------------------------
loss: 2.295084  [   64/60000]
loss: 2.296698  [ 6464/60000]
loss: 2.296507  [12864/60000]
loss: 2.304253  [19264/60000]
loss: 2.280333  [25664/60000]
loss: 2.302902  [32064/60000]
loss: 2.292485  [38464/60000]
loss: 2.297485  [44864/60000]
loss: 2.295645  [51264/60000]
loss: 2.295810  [57664/60000]
Test Error: 
 Accuracy: 23.5%, Avg loss: 2.293232 

Epoch 3
-------------------------------
loss: 2.290836  [   64/60000]
loss: 2.291675  [ 6464/60000]
loss: 2.290273  [12864/60000]
loss: 2.298427  [19264/60000]
loss: 2.276127  [25664/60000]
loss: 2.293890  [32064/600

## Setting custom kernels

Just like with model weights, we can set specific kernels for CNNs

In [43]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # how many dimensions does the bias vector of self.conv1 have?
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1) 
    # stride = 1, padding = 0, dilation = 1 by default
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()

    with torch.no_grad():
      weights = torch.FloatTensor([
          [0, 1, 0, 1, 0],
          [1, 0, 1, 0, 1],
          [1, 0, 1, 0, 1],
          [0, 0, 1, 0, 1],
          [1, 1, 1, 1, 0]
      ])
      weights = weights.view(1, 1, 5, 5).repeat(6, 1, 1, 1)
      # here we have 6 copies of the same kernel
      # as you can imagine, you must initialise each kernel
      # as you please. It doesn't have to be the same kernel for
      # each channel

      # why do we need 6 kernels for conv1?
      self.conv1.weight = nn.Parameter(weights, requires_grad=True)

  def forward(self, x):
    # input size = 1x28x28
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()

In [44]:
net.conv1.weight, net.conv1.bias

(Parameter containing:
 tensor([[[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1., 0., 1.],
           [1., 1., 1., 1., 0.]]],
 
 
         [[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1., 0., 1.],
           [1., 1., 1., 1., 0.]]],
 
 
         [[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1., 0., 1.],
           [1., 1., 1., 1., 0.]]],
 
 
         [[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1., 0., 1.],
           [1., 1., 1., 1., 0.]]],
 
 
         [[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1., 0., 1.],
           [1., 1., 1., 1., 0.]]],
 
 
         [[[0., 1., 0., 1., 0.],
           [1., 0., 1., 0., 1.],
           [1., 0., 1., 0., 1.],
           [0., 0., 1.,

In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, net, loss_fn, optimizer)
  test_loop(test_dataloader, net, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.269511  [   64/60000]
loss: 2.114079  [ 6464/60000]
loss: 1.767052  [12864/60000]
loss: 1.519192  [19264/60000]
loss: 1.261594  [25664/60000]
loss: 1.148371  [32064/60000]
loss: 1.074535  [38464/60000]
loss: 0.979699  [44864/60000]
loss: 1.025754  [51264/60000]
loss: 0.976575  [57664/60000]
Test Error: 
 Accuracy: 63.8%, Avg loss: 0.961789 

Epoch 2
-------------------------------
loss: 0.958201  [   64/60000]
loss: 1.026418  [ 6464/60000]
loss: 0.794304  [12864/60000]
loss: 0.971024  [19264/60000]
loss: 0.876761  [25664/60000]
loss: 0.881821  [32064/60000]
loss: 0.875509  [38464/60000]
loss: 0.823083  [44864/60000]
loss: 0.844957  [51264/60000]
loss: 0.910138  [57664/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 0.859914 

Epoch 3
-------------------------------
loss: 0.805608  [   64/60000]
loss: 0.939797  [ 6464/60000]
loss: 0.666275  [12864/60000]
loss: 0.891263  [19264/60000]
loss: 0.839616  [25664/60000]
loss: 0.797984  [32064/600

In [40]:
net.conv1.weight

Parameter containing:
tensor([[[[0.0080, 1.0123, 0.0096, 1.0089, 0.0064],
          [1.0093, 0.0117, 1.0085, 0.0069, 1.0052],
          [1.0089, 0.0104, 1.0074, 0.0053, 1.0039],
          [0.0086, 0.0102, 1.0072, 0.0047, 1.0035],
          [1.0084, 1.0096, 1.0061, 1.0036, 0.0028]]],


        [[[0.0057, 1.0073, 0.0047, 1.0062, 0.0047],
          [1.0068, 0.0078, 1.0057, 0.0062, 1.0044],
          [1.0064, 0.0073, 1.0050, 0.0053, 1.0031],
          [0.0055, 0.0071, 1.0052, 0.0051, 1.0029],
          [1.0055, 1.0073, 1.0053, 1.0053, 0.0027]]],


        [[[0.0088, 1.0132, 0.0111, 1.0125, 0.0094],
          [1.0105, 0.0135, 1.0112, 0.0123, 1.0094],
          [1.0107, 0.0133, 1.0104, 0.0113, 1.0082],
          [0.0104, 0.0135, 1.0102, 0.0105, 1.0075],
          [1.0111, 1.0133, 1.0091, 1.0093, 0.0070]]],


        [[[0.0094, 1.0123, 0.0096, 1.0083, 0.0031],
          [1.0107, 0.0119, 1.0096, 0.0079, 1.0037],
          [1.0097, 0.0109, 1.0089, 0.0070, 1.0032],
          [0.0086, 0.0108, 1.0

### Freezing a specific kernel for training

Similarly to a dense neural network, we can freeze the kernels and train. Let's see what happens

In [48]:
net = Net()
net.conv1.weight.requires_grad_(False)

Parameter containing:
tensor([[[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]

In [47]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, net, loss_fn, optimizer)
  test_loop(test_dataloader, net, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.332624  [   64/60000]
loss: 2.237638  [ 6464/60000]
loss: 2.086502  [12864/60000]
loss: 1.941406  [19264/60000]
loss: 1.584647  [25664/60000]
loss: 1.419474  [32064/60000]
loss: 1.275614  [38464/60000]
loss: 1.137595  [44864/60000]
loss: 1.144067  [51264/60000]
loss: 1.013160  [57664/60000]
Test Error: 
 Accuracy: 62.8%, Avg loss: 1.010004 

Epoch 2
-------------------------------
loss: 1.058013  [   64/60000]
loss: 1.042434  [ 6464/60000]
loss: 0.810853  [12864/60000]
loss: 0.986306  [19264/60000]
loss: 0.898038  [25664/60000]
loss: 0.935231  [32064/60000]
loss: 0.900562  [38464/60000]
loss: 0.837594  [44864/60000]
loss: 0.875587  [51264/60000]
loss: 0.868203  [57664/60000]
Test Error: 
 Accuracy: 67.3%, Avg loss: 0.859339 

Epoch 3
-------------------------------
loss: 0.867578  [   64/60000]
loss: 0.896618  [ 6464/60000]
loss: 0.670783  [12864/60000]
loss: 0.892644  [19264/60000]
loss: 0.847780  [25664/60000]
loss: 0.851791  [32064/600

In [44]:
net.conv1.weight

Parameter containing:
tensor([[[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 0., 1., 0.],
          [1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1.],
          [0., 0., 1., 0., 1.],
          [1., 1., 1., 1., 0.]